In [13]:
import pandas as pd
import numpy as np
import pyreadr as pyr
import sklearn as sk
import matplotlib.pyplot as plt
import scipy as sc
import os
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from scipy.stats import chi2
from sklearn.metrics import make_scorer
from numpy.random import normal
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from random import choices
from sklearn.model_selection import train_test_split
from mlens.ensemble import SuperLearner
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE 
from sklearn.metrics import confusion_matrix
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from scikeras.wrappers import KerasClassifier
from keras.utils import np_utils

In [2]:
#Set directory:
os.chdir("C:/Users/hp/OneDrive - Universidad de los Andes/Documentos/Docs/Universidad/2022-2/Big Data/Trabajo Final")

In [3]:
df=pd.read_csv("Base_2016_Clean.csv",low_memory=False)

In [4]:
##Imputar missing values
imputer=KNNImputer(n_neighbors=19,weights='distance')
df_imp=imputer.fit_transform(df)
X=pd.DataFrame(df_imp,columns=df.columns)
Y=X["voto_ofrecido"]
X=X[[i for i in X.columns if i!="llaveper_n16" and i!="voto_ofrecido"]]

In [5]:
## Train y test ####
X_train,X_test, y_train, y_test = train_test_split(X,Y,stratify=Y,test_size=0.2,random_state=911)
##SMOTE para la train
oversample = SMOTE(sampling_strategy=0.8,k_neighbors=19,random_state=911,n_jobs=-1)
X_trains, y_trains = oversample.fit_resample(X_train, y_train)

c:\Users\hp\miniconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [ ]:
#Redondeo
lista=['t_cuartos_hogar','t_personas', 'min_cabec','n_televisores', 'n_computadores', 'edad',
'valor_arriendo_pagado', 'hor_salud', 'n_duchas', 'n_otros_bienes']
for i in X_trains.columns:
    if i not in lista:
        X_trains[i]=X_trains[i].round(0)
for i in X_train.columns:
    if i not in lista:
        X_train[i]=X_train[i].round(0)
for i in X_test.columns:
    if i not in lista:
        X_test[i]=X_test[i].round(0)

In [ ]:
ds=(df[['voto_ofrecido','edad','sexo','zona', 'nivel_educ_1','nivel_educ_2','nivel_educ_3','nivel_educ_4','sp_estrato_2',
'sp_estrato_3', 'sp_estrato_4', 'sp_estrato_5','sp_estrato_6','familias_accion', 'tenencia_vivienda_1',
 'tenencia_vivienda_2', 'tenencia_vivienda_3', 'tenencia_vivienda_4',
 'tenencia_vivienda_5','org_sindicato', 
'voto_alcaldia', 'evitar_iva']].describe(include="all"))
ds=ds.T
ds=ds[["count", "mean", "std", "50%"]]
ds=ds.round(4)
print(ds.to_latex())

In [11]:
##Metrica
def wmir(y_true,y_pred):
    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    # False negative rate
    FNR = FN/(TP+FN)
    #False positive rate
    FPR = FP/(FP+TN)
    penalty=FNR*(2/3)+(1/3)*FPR
    return penalty

wmirs=make_scorer(wmir, greater_is_better=False)

def FF(y_true,y_pred):
    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    # False negative rate
    FNR = FN/(TP+FN)
    FPR = FP/(FP+TN)
    return FNR,FPR

In [ ]:
# Modelos

In [41]:
knn=KNeighborsClassifier()
neighbors=[11,13,15,17,19,21,23,25,27,29,31]
params = {
    'n_neighbors': neighbors,
    'weights': ['uniform','distance'],
}
knn_cv=GridSearchCV(knn,params,scoring=wmirs,n_jobs=-1, cv=10)
re_knncv=knn_cv.fit(X_trains,y_trains.values.ravel())

In [42]:
re_knncv.best_params_

{'n_neighbors': 11, 'weights': 'distance'}

In [43]:
print(wmir(y_test, re_knncv.predict(X_test)),FF(y_test, re_knncv.predict(X_test)))
print(wmir(y_trains, re_knncv.predict(X_trains)),FF(y_trains, re_knncv.predict(X_trains))) 

0.32088659639680045 (0.14423076923076922, 0.674198250728863)
0.0 (0.0, 0.0)


In [44]:
parameters={'solver':['newton-cg', 'lbfgs', 'sag', 'saga'],'warm_start': [True,False]}
logistica=LogisticRegression(fit_intercept=True, penalty="none", random_state=911, max_iter=1000,n_jobs=-1)
logcv=GridSearchCV(logistica, parameters, scoring=wmirs)
resultslogistic_b=logcv.fit(X_trains, y_trains.values.ravel())

resultslogistic_b.best_params_

{'solver': 'lbfgs', 'warm_start': True}

In [56]:
print(wmir(y_test.values.ravel(),resultslogistic_b.predict(X_test)),FF(y_test.values.ravel(),resultslogistic_b.predict(X_test)))
print(wmir(y_trains.values.ravel(),resultslogistic_b.predict(X_trains)),FF(y_trains.values.ravel(),resultslogistic_b.predict(X_trains)))

0.41224676683860356 (0.4951923076923077, 0.24635568513119532)
0.3159230548711512 (0.3571753986332574, 0.23341836734693877)


In [46]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

grilla=np.linspace(0,1,100)
grilla=grilla.tolist()
reg_param={"reg_param": grilla}
qda=QuadraticDiscriminantAnalysis()
qdacv=GridSearchCV(qda, reg_param, scoring=wmirs,n_jobs=-1, cv=10)
modelqda=qdacv.fit(X_trains, y_trains.values.ravel())

modelqda.best_params_

c:\Users\hp\miniconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


{'reg_param': 0.020202020202020204}

In [47]:
print(wmir(y_test.values.ravel(),modelqda.predict(X_test)),FF(y_test.values.ravel(),modelqda.predict(X_test)))
print(wmir(y_trains.values.ravel(),modelqda.predict(X_trains)),FF(y_trains.values.ravel(),modelqda.predict(X_trains)))

0.4259549973835688 (0.4807692307692308, 0.3163265306122449)
0.13648363185169934 (0.06651480637813212, 0.2764212827988338)


In [335]:
qda2=QuadraticDiscriminantAnalysis(reg_param=0.425)
modelqda2=qda2.fit(X_trains, y_trains.values.ravel())

c:\Users\hp\miniconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [336]:
print(wmir(y_test.values.ravel(),modelqda2.predict(X_test)),FF(y_test.values.ravel(),modelqda2.predict(X_test)))
print(wmir(y_trains.values.ravel(),modelqda2.predict(X_trains)),FF(y_trains.values.ravel(),modelqda2.predict(X_trains)))

0.34630335650743815 (0.25961538461538464, 0.5196793002915452)
0.2370392999373521 (0.11503416856492027, 0.48104956268221577)


In [238]:
lda=LinearDiscriminantAnalysis(solver='lsqr',shrinkage=0.525,n_components=1)
modellda=lda.fit(X_trains, y_trains.values.ravel())

In [239]:
print(wmir(y_test.values.ravel(),modellda.predict(X_test)),FF(y_test.values.ravel(),modellda.predict(X_test)))
print(wmir(y_trains.values.ravel(),modellda.predict(X_trains)),FF(y_trains.values.ravel(),modellda.predict(X_trains)))

0.36543133737011285 (0.42788461538461536, 0.24052478134110788)
0.33815520187899434 (0.38724373576309795, 0.23997813411078717)


In [54]:
l1 = np.linspace(0,1,10)
C = np.linspace(0,1,10)
parameters={'solver':['saga'],
            'warm_start': [True,False],
            'l1_ratio': l1,
            'C': C
            }
elastic=LogisticRegression(fit_intercept=True, penalty="elasticnet", random_state=911, max_iter=1000,n_jobs=-1)
encv=GridSearchCV(elastic, parameters, scoring=wmirs)
resultsen=encv.fit(X_trains, y_trains.values.ravel())

resultsen.best_params_

c:\Users\hp\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "c:\Users\hp\miniconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 428, in _process_worker
    r = call_item()
  File "c:\Users\hp\miniconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 275, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "c:\Users\hp\miniconda3\lib\site-packages\joblib\_parallel_backends.py", line 620, in __call__
    return 

{'C': 0.4444444444444444,
 'l1_ratio': 0.5555555555555556,
 'solver': 'saga',
 'warm_start': True}

In [55]:
print(wmir(y_test.values.ravel(),resultsen.predict(X_test)),FF(y_test.values.ravel(),resultsen.predict(X_test)))
print(wmir(y_trains.values.ravel(),resultsen.predict(X_trains)),FF(y_trains.values.ravel(),resultsen.predict(X_trains)))

0.4065653734021081 (0.4855769230769231, 0.24854227405247814)
0.3194462523050222 (0.3630979498861048, 0.23214285714285715)


In [303]:
elasticnw=LogisticRegression(fit_intercept=True,l1_ratio=1,C=0.025, penalty="elasticnet",solver='saga', random_state=911, max_iter=1000,n_jobs=-1)
modelenw=elasticnw.fit(X_trains, y_trains.values.ravel())

In [304]:
print(wmir(y_test.values.ravel(),modelenw.predict(X_test)),FF(y_test.values.ravel(),modelenw.predict(X_test)))
print(wmir(y_trains.values.ravel(),modelenw.predict(X_trains)),FF(y_trains.values.ravel(),modelenw.predict(X_trains)))

0.3635344247589145 (0.4326923076923077, 0.22521865889212828)
0.3485430211785332 (0.4123006833712984, 0.2210276967930029)


In [209]:
from sklearn.ensemble import RandomForestClassifier

RF=RandomForestClassifier(n_estimators=50,max_depth=2,criterion='entropy',class_weight={0: 1,1: 3.6},max_features=None,bootstrap=True,n_jobs=-1,random_state=911)
resultsRF=RF.fit(X_trains, y_trains.values.ravel())

In [210]:
print(wmir(y_test.values.ravel(),resultsRF.predict(X_test)),FF(y_test.values.ravel(),resultsRF.predict(X_test)))
print(wmir(y_trains.values.ravel(),resultsRF.predict(X_trains)),FF(y_trains.values.ravel(),resultsRF.predict(X_trains)))

0.3244374672946101 (0.1346153846153846, 0.7040816326530612)
0.2643676767810932 (0.04943052391799545, 0.6942419825072886)


In [517]:
import xgboost as xgb

XGB=xgb.XGBRFClassifier(n_estimators=50,max_depth=2,scale_pos_weight=3.55,objective='binary:logistic',booster='gbtree',grow_policy='lossguide',n_jobs=-1,random_state=911)
resultsXGB=XGB.fit(X_trains, y_trains.values.ravel())

In [518]:
print(wmir(y_test.values.ravel(),resultsXGB.predict(X_test)),FF(y_test.values.ravel(),resultsXGB.predict(X_test)))
print(wmir(y_trains.values.ravel(),resultsXGB.predict(X_trains)),FF(y_trains.values.ravel(),resultsXGB.predict(X_trains)))

0.32631569111160946 (0.052884615384615384, 0.8731778425655977)
0.29637475068126823 (0.013895216400911162, 0.8613338192419825)


In [613]:
XGB2=xgb.XGBClassifier(n_estimators=35,max_depth=3,scale_pos_weight=16.75,reg_alpha=0,reg_lambda=1,objective='binary:logistic',booster='gbtree',grow_policy='depthwise',n_jobs=-1,random_state=911)
resultsXGB2=XGB2.fit(X_trains, y_trains.values.ravel())

In [614]:
print(wmir(y_test.values.ravel(),resultsXGB2.predict(X_test)),FF(y_test.values.ravel(),resultsXGB2.predict(X_test)))
print(wmir(y_trains.values.ravel(),resultsXGB2.predict(X_trains)),FF(y_trains.values.ravel(),resultsXGB2.predict(X_trains)))

0.315392090902295 (0.15384615384615385, 0.6384839650145773)
0.20405147089750314 (0.003416856492027335, 0.6053206997084548)


In [725]:
ANN=MLPClassifier(activation='relu',solver='adam',alpha=0.58,max_iter=50,verbose=True,hidden_layer_sizes=(2,4,8,16,32,64),random_state=911)
resultsANN=ANN.fit(X_trains, y_trains.values.ravel())

Iteration 1, loss = 0.80004081
Iteration 2, loss = 0.74373039
Iteration 3, loss = 0.69466628
Iteration 4, loss = 0.66457555
Iteration 5, loss = 0.64645297
Iteration 6, loss = 0.63446486
Iteration 7, loss = 0.62376334
Iteration 8, loss = 0.61568682
Iteration 9, loss = 0.60971159
Iteration 10, loss = 0.60361828
Iteration 11, loss = 0.60040259
Iteration 12, loss = 0.59316720
Iteration 13, loss = 0.59003168
Iteration 14, loss = 0.58573252
Iteration 15, loss = 0.57939302
Iteration 16, loss = 0.58224062
Iteration 17, loss = 0.57436364
Iteration 18, loss = 0.57035829
Iteration 19, loss = 0.56819288
Iteration 20, loss = 0.56632097
Iteration 21, loss = 0.56593449
Iteration 22, loss = 0.56100072
Iteration 23, loss = 0.55827040
Iteration 24, loss = 0.55869989
Iteration 25, loss = 0.55761263
Iteration 26, loss = 0.55600046
Iteration 27, loss = 0.55556500
Iteration 28, loss = 0.55318957
Iteration 29, loss = 0.55478939
Iteration 30, loss = 0.55502493
Iteration 31, loss = 0.55226768
Iteration 32, los

c:\Users\hp\miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [726]:
print(wmir(y_test.values.ravel(),resultsANN.predict(X_test)),FF(y_test.values.ravel(),resultsANN.predict(X_test)))
print(wmir(y_trains.values.ravel(),resultsANN.predict(X_trains)),FF(y_trains.values.ravel(),resultsANN.predict(X_trains)))

0.37792479629214315 (0.41346153846153844, 0.30685131195335275)
0.2287538380142164 (0.20318906605922551, 0.27988338192419826)


In [ ]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
param = {'num_leaves': 31, 'objective': 'binary'}
train_data = lgb.Dataset(X_trains, label=y_trains)
param['metric'] = 'auc'
lgbclas=LGBMClassifier(n_estimators=106,scale_pos_weight=19,boosting_type='gbdt',reg_lambda=8 ,reg_alpha=9,num_leaves=9,max_depth=5,random_state=911,n_jobs=-1)
re_lgb=lgbclas.fit(X_trains,y_trains,eval_metric=wmir)
print(wmir(y_trains,re_lgb.predict(X_trains)))
print(FF(y_trains,re_lgb.predict(X_trains)))
print(wmir(y_test,re_lgb.predict(X_test)))
print(FF(y_test,re_lgb.predict(X_test)))
print(accuracy_score(y_test,re_lgb.predict(X_test)))

In [ ]:
neurons=[(2,4,8,16,32,64),(1,2,4,8,16,32)]
activation=[('sigmoid','sigmoid','sigmoid','sigmoid','sigmoid','sigmoid'),('sigmoid','tanh','sigmoid','tanh','sigmoid','sigmoid'),
    ('sigmoid','relu','sigmoid','relu','sigmoid','sigmoid'),
    ('tanh','tanh','relu','tanh','tanh','tanh'),
    ('sigmoid','sigmoid','softmax','sigmoid','sigmoid','sigmoid')]
dropout=[(0.5,0.5,0.5,0.5,0.5),(0.1,0.1,0.1,0.1,0.1)]
batch_size=[128]

param_grid=dict(model_activation=activation,modelneurons=neurons,model_dropout=dropout, batch_size=batch_size)

def model(neurons=(1,2,4,8,16,32), activation=('sigmoid','sigmoid','sigmoid','sigmoid','sigmoid','sigmoid'), dropout=(0.1,0.1,0.1,0.1,0.1)):
    model = Sequential()
    model.add(Dense(neurons[0], input_shape=(137,), activation=activation[0]))
    model.add(Dense(neurons[1], activation=activation[1]))
    model.add(Dropout(dropout[0]))
    model.add(Dense(neurons[2], activation=activation[2]))
    model.add(Dropout(dropout[1]))
    model.add(Dense(neurons[3], activation=activation[3]))
    model.add(Dropout(dropout[2]))
    model.add(Dense(neurons[4], activation=activation[4]))
    model.add(Dropout(dropout[3]))
    model.add(Dense(neurons[5], activation=activation[5]))
    model.add(Dropout(dropout[4]))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='FalseNegatives')
    return model
modelo=KerasClassifier(model=model,epochs=80,verbose=1)
grid=GridSearchCV(estimator=modelo, param_grid=param_grid, scoring=wmir, n_jobs=-1, cv=5, verbose=1)
re_ann=grid.fit(X_trains,y_trains, class_weight={0: 1, 1: 2.8})
print(re_ann.best_params_)
y_pred=re_ann.predict(X_trains)
y_pred2=re_ann.predict(X_test)